In [1]:
# uncomment and run if mismo is not installed
# %pip install -q git+https://github.com/NickCrews/mismo@main

In [1]:
from __future__ import annotations

import ibis
from ibis import _
from ibis.backends.duckdb import Backend as DuckDBBackend

import mismo

ibis.options.interactive = True

In [2]:
conn: DuckDBBackend = ibis.duckdb.connect()
clean = conn.read_parquet("../../data/alaska-addresses-clean.parquet")
dirty = conn.read_parquet("../../data/alaska-addresses-dirty.parquet")
print(clean.count().execute())
clean

315659


┏━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ record_id ┃ state  ┃ city         ┃ postcode ┃ street                 ┃ unit   ┃ lon         ┃ lat       ┃
┡━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━┩
│ int64     │ string │ string       │ string   │ string                 │ string │ float64     │ float64   │
├───────────┼────────┼──────────────┼──────────┼────────────────────────┼────────┼─────────────┼───────────┤
│         0 │ AK     │ ANCHOR POINT │ 99556    │ 23837 NIKOLAI STREET   │ NULL   │ -151.746357 │ 59.926617 │
│         1 │ AK     │ ANCHOR POINT │ 99556    │ 24338 MISTY LANE       │ NULL   │ -151.739071 │ 59.919594 │
│         2 │ AK     │ ANCHOR POINT │ 99556    │ 24715 STERLING HIGHWAY │ NULL   │ -151.755646 │ 59.913984 │
│         3 │ AK     │ ANCHOR POINT │ 99556    │ 24803 INLET COVE LANE  │ NULL   │ -151.757118 │ 59.912571 │
│         4 │ AK     │ ANCHOR POINT │ 99556    │ 24825 INLET COVE LANE  │ NULL   │ -151.757657 │ 59.912205 │
│         5 │ AK     │ ANCHOR POINT │ 99556    │ 24845 INLET COVE LANE  │ NULL   │ -151.757918 │ 59.911979 │
│         6 │ AK     │ ANCHOR POINT │ 99556    │ 24865 INLET COVE LANE  │ NULL   │ -151.758861 │ 59.911740 │
│         7 │ AK     │ ANCHOR POINT │ 99556    │ 25100 PEPPER ROAD      │ NULL   │ -151.709791 │ 59.908129 │
│         8 │ AK     │ ANCHOR POINT │ 99556    │ 25238 STINKBUG STREET  │ NULL   │ -151.692157 │ 59.906734 │
│         9 │ AK     │ ANCHOR POINT │ 99556    │ 25405 STINKBUG STREET  │ NULL   │ -151.695885 │ 59.903846 │
│         … │ …      │ …            │ …        │ …                      │ …      │           … │         … │
└───────────┴────────┴──────────────┴──────────┴────────────────────────┴────────┴─────────────┴───────────┘

In [3]:
print(dirty.count().execute())
dirty

992329


┏━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ record_id ┃ state  ┃ city     ┃ postcode ┃ street             ┃ unit   ┃
┡━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ int64     │ string │ string   │ string   │ string             │ string │
├───────────┼────────┼──────────┼──────────┼────────────────────┼────────┤
│         0 │ AK     │ ACHORAGE │ 99417    │ 2010 WLEY HARI AVE │ NULL   │
│         1 │ AK     │ ADAK     │ 99546    │ 241 AMCHITKA CIR   │ NULL   │
│         2 │ AK     │ ADAK     │ 99546    │ 24317 SEWARD HWY   │ NULL   │
│         3 │ AK     │ ADAK     │ 99546    │ 351 AURIGA LN      │ NULL   │
│         4 │ AK     │ AKIACHAK │ 99551    │ 12410 HOPA CIR     │ NULL   │
│         5 │ AK     │ AKIACHAK │ 99551    │ 12812 OOMIPK CIR   │ NULL   │
│         6 │ AK     │ AKIACHAK │ 99551    │ 1760 A HEIRLAM DR  │ NULL   │
│         7 │ AK     │ AKIACHAK │ 99551    │ 1985 BRANDILYN ST  │ NULL   │
│         8 │ AK     │ AKIACHAK │ 99551    │ 2503 LN HONDA DR   │ NULL   │
│         9 │ AK     │ AKIACHAK │ 99551    │ 8731 SULTANA DR    │ NULL   │
│         … │ …      │ …        │ …        │ …                  │ …      │
└───────────┴────────┴──────────┴──────────┴────────────────────┴────────┘

In [4]:
def norm_string(s: ibis.ir.StringValue) -> ibis.ir.StringValue:
    return mismo.text.norm_whitespace(s).upper().nullif("")


normed_strings = ibis.selectors.across(ibis.selectors.of_type(str), norm_string)

clean = clean.mutate(normed_strings).distinct().cache()
dirty = dirty.mutate(normed_strings).cache()

In [7]:
exact_linker = mismo.KeyLinker(("city", "postcode", "street", "unit"))
exact_linker.pair_counts(clean, dirty).chart()

AttributeError: 'Table' object has no attribute '_KEY_COUNTS_SPEC'

In [10]:
many_linkages = [
    mismo.KeyLinker(("city", "street", "unit"))(clean, dirty),
    mismo.KeyLinker(("postcode", "street", "unit"))(clean, dirty),
]
linkage = mismo.linkage.intersect(many_linkages)
linkage

In [11]:
linkage.links

┏━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ record_id_l ┃ state_l ┃ city_l          ┃ postcode_l ┃ street_l                ┃ unit_l ┃ lon_l       ┃ lat_l     ┃ record_id_r ┃ state_r ┃ city_r          ┃ postcode_r ┃ street_r                ┃ unit_r ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ int64       │ string  │ string          │ string     │ string                  │ string │ float64     │ float64   │ int64       │ string  │ string          │ string     │ string                  │ string │
├─────────────┼─────────┼─────────────────┼────────────┼─────────────────────────┼────────┼─────────────┼───────────┼─────────────┼─────────┼─────────────────┼────────────┼─────────────────────────┼────────┤
│      166795 │ AK      │ FORT WAINWRIGHT │ 99703      │ 4409 CABRUAN HILLS LOOP │ UNIT 3 │ -147.697064 │ 64.829728 │      711127 │ AK      │ FORT WAINWRIGHT │ 99703      │ 4409 CABRUAN HILLS LOOP │ UNIT 3 │
│      166795 │ AK      │ FORT WAINWRIGHT │ 99703      │ 4409 CABRUAN HILLS LOOP │ UNIT 3 │ -147.697064 │ 64.829728 │      711151 │ AK      │ FORT WAINWRIGHT │ 99703      │ 4409 CABRUAN HILLS LOOP │ UNIT 3 │
│      167112 │ AK      │ FORT WAINWRIGHT │ 99703      │ 4747 SOAPBERRY LOOP     │ UNIT 2 │ -147.662568 │ 64.823829 │      711168 │ AK      │ FORT WAINWRIGHT │ 99703      │ 4747 SOAPBERRY LOOP     │ UNIT 2 │
│      167112 │ AK      │ FORT WAINWRIGHT │ 99703      │ 4747 SOAPBERRY LOOP     │ UNIT 2 │ -147.662568 │ 64.823829 │      711169 │ AK      │ FORT WAINWRIGHT │ 99703      │ 4747 SOAPBERRY LOOP     │ UNIT 2 │
│      167112 │ AK      │ FORT WAINWRIGHT │ 99703      │ 4747 SOAPBERRY LOOP     │ UNIT 2 │ -147.662568 │ 64.823829 │      711183 │ AK      │ FORT WAINWRIGHT │ 99703      │ 4747 SOAPBERRY LOOP     │ UNIT 2 │
│      157303 │ AK      │ FAIRBANKS       │ 99709      │ 570 RIVERSTONE WAY      │ UNIT 3 │ -147.866477 │ 64.846606 │      685815 │ AK      │ FAIRBANKS       │ 99709      │ 570 RIVERSTONE WAY      │ UNIT 3 │
│      166763 │ AK      │ FORT WAINWRIGHT │ 99703      │ 4401 CABRUAN HILLS LOOP │ UNIT 6 │ -147.696731 │ 64.829329 │      711042 │ AK      │ FORT WAINWRIGHT │ 99703      │ 4401 CABRUAN HILLS LOOP │ UNIT 6 │
│      166795 │ AK      │ FORT WAINWRIGHT │ 99703      │ 4409 CABRUAN HILLS LOOP │ UNIT 3 │ -147.697064 │ 64.829728 │      711107 │ AK      │ FORT WAINWRIGHT │ 99703      │ 4409 CABRUAN HILLS LOOP │ UNIT 3 │
│      166795 │ AK      │ FORT WAINWRIGHT │ 99703      │ 4409 CABRUAN HILLS LOOP │ UNIT 3 │ -147.697064 │ 64.829728 │      711110 │ AK      │ FORT WAINWRIGHT │ 99703      │ 4409 CABRUAN HILLS LOOP │ UNIT 3 │
│      166795 │ AK      │ FORT WAINWRIGHT │ 99703      │ 4409 CABRUAN HILLS LOOP │ UNIT 3 │ -147.697064 │ 64.829728 │      711122 │ AK      │ FORT WAINWRIGHT │ 99703      │ 4409 CABRUAN HILLS LOOP │ UNIT 3 │
│           … │ …       │ …               │ …          │ …                       │ …      │           … │         … │           … │ …       │ …               │ …          │ …                       │ …      │
└─────────────┴─────────┴─────────────────┴────────────┴─────────────────────────┴────────┴─────────────┴───────────┴─────────────┴─────────┴─────────────────┴────────────┴─────────────────────────┴────────┘

In [12]:
linkage.__sublinkages__()

(Linkage<left=315_659, right=992_329, links=102>,
 Linkage<left=315_659, right=992_329, links=102>)

In [14]:
linkage2 = mismo.linkage.union(
    [linkage, mismo.KeyLinker(("city", _.street[:5], "unit"))(clean, dirty)]
)
linkage2

TypeError: ('received some non-Linkages: [<mismo.linkage._combine.IntersectionLinkage object at 0x1296f80b0>]', [<mismo.linkage._combine.IntersectionLinkage object at 0x1296f80b0>])

In [10]:
linkage2.__sublinkages__()

(<mismo.linkage._combine.AndJoinConditionsLinkage at 0x128866bd0>,
 KeyLinkage(
     keys=(city, _.street[slice(None, 5, None)], unit)
     nleft=315_659
     nright=992_329
     nlinks=3_417
 ))

In [ ]:
linkage = exact_linker.linkage(clean, dirty)
linkage

KeyLinkage(
    keys=(city, postcode, street, unit)
    nleft=315_659
    nright=992_329
    nlinks=102
)

In [7]:
is_laurel = _.street.contains("4228 LAUREL")
dirty.filter(is_laurel), clean.filter(is_laurel)

(┏━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━┓
 ┃ record_id ┃ state  ┃ city      ┃ postcode ┃ street         ┃ unit   ┃
 ┡━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━┩
 │ int64     │ string │ string    │ string   │ string         │ string │
 ├───────────┼────────┼───────────┼──────────┼────────────────┼────────┤
 │    354101 │ AK     │ ANCHORAGE │ 99508    │ 4228 LAUREL ST │ NULL   │
 │    354102 │ AK     │ ANCHORAGE │ 99508    │ 4228 LAUREL ST │ NULL   │
 │    354103 │ AK     │ ANCHORAGE │ 99508    │ 4228 LAUREL ST │ NULL   │
 │    354104 │ AK     │ ANCHORAGE │ 99508    │ 4228 LAUREL ST │ NULL   │
 │    354105 │ AK     │ ANCHORAGE │ 99508    │ 4228 LAUREL ST │ NULL   │
 │    354106 │ AK     │ ANCHORAGE │ 99508    │ 4228 LAUREL ST │ NULL   │
 │    354107 │ AK     │ ANCHORAGE │ 99508    │ 4228 LAUREL ST │ NULL   │
 │    354108 │ AK     │ ANCHORAGE │ 99508    │ 4228 LAUREL ST │ NULL   │
 │    354109 │ AK     │ ANCHORAGE │ 99508    │ 4228

In [8]:
def value_trigrams(s: ibis.ir.StringValue) -> ibis.ir.ArrayColumn:
    return (
        mismo.text.tokenize(s).map(lambda token: mismo.text.ngrams(token, 3)).flatten()
    )


def trigrams(t: ibis.Table) -> ibis.ir.ArrayColumn:
    return (
        ibis.array(
            [
                value_trigrams(t.city),
                value_trigrams(t.postcode),
                value_trigrams(t.street),
            ]
        )
        .flatten()
        .unique()
        .name("trigrams")
    )


clean = clean.mutate(trigrams(clean)).cache()
dirty = dirty.mutate(trigrams(dirty)).cache()
clean

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┏━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ record_id ┃ state  ┃ city         ┃ postcode ┃ street                 ┃ unit   ┃ lon         ┃ lat       ┃ trigrams                ┃
┡━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ int64     │ string │ string       │ string   │ string                 │ string │ float64     │ float64   │ array<string>           │
├───────────┼────────┼──────────────┼──────────┼────────────────────────┼────────┼─────────────┼───────────┼─────────────────────────┤
│         0 │ AK     │ ANCHOR POINT │ 99556    │ 23837 NIKOLAI STREET   │ NULL   │ -151.746357 │ 59.926617 │ ['STR', 'NIK', ... +20] │
│         1 │ AK     │ ANCHOR POINT │ 99556    │ 24338 MISTY LANE       │ NULL   │ -151.739071 │ 59.919594 │ ['ANE', 'LAN', ... +16] │
│         2 │ AK     │ ANCHOR POINT │ 99556    │ 24715 STERLING HIGHWAY │ NULL   │ -151.755646 │ 59.913984 │ ['WAY', 'GHW', ... +22] │
│         3 │ AK     │ ANCHOR POINT │ 99556    │ 24803 INLET COVE LANE  │ NULL   │ -151.757118 │ 59.912571 │ ['ANE', 'LAN', ... +18] │
│         4 │ AK     │ ANCHOR POINT │ 99556    │ 24825 INLET COVE LANE  │ NULL   │ -151.757657 │ 59.912205 │ ['ANE', 'LAN', ... +18] │
│         5 │ AK     │ ANCHOR POINT │ 99556    │ 24845 INLET COVE LANE  │ NULL   │ -151.757918 │ 59.911979 │ ['ANE', 'LAN', ... +18] │
│         6 │ AK     │ ANCHOR POINT │ 99556    │ 24865 INLET COVE LANE  │ NULL   │ -151.758861 │ 59.911740 │ ['ANE', 'LAN', ... +18] │
│         7 │ AK     │ ANCHOR POINT │ 99556    │ 25100 PEPPER ROAD      │ NULL   │ -151.709791 │ 59.908129 │ ['OAD', 'ROA', ... +17] │
│         8 │ AK     │ ANCHOR POINT │ 99556    │ 25238 STINKBUG STREET  │ NULL   │ -151.692157 │ 59.906734 │ ['BUG', 'TIN', ... +21] │
│         9 │ AK     │ ANCHOR POINT │ 99556    │ 25405 STINKBUG STREET  │ NULL   │ -151.695885 │ 59.903846 │ ['STR', 'BUG', ... +21] │
│         … │ …      │ …            │ …        │ …                      │ …      │           … │         … │ …                       │
└───────────┴────────┴──────────────┴──────────┴────────────────────────┴────────┴─────────────┴───────────┴─────────────────────────┘

In [35]:
easy_linkage = mismo.KeyLinker("state", "city", "postcode", "street")
easy_linkage.pair_counts(clean, dirty)

┏━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃ state  ┃ city      ┃ postcode ┃ street                ┃ n     ┃
┡━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━┩
│ string │ string    │ string   │ string                │ int64 │
├────────┼───────────┼──────────┼───────────────────────┼───────┤
│ AK     │ ANCHORAGE │ 99503    │ 1003 CHUGACH WAY      │  4107 │
│ AK     │ ANCHORAGE │ 99515    │ 2200 MINERVA WAY      │  3120 │
│ AK     │ FAIRBANKS │ 99701    │ 1516 GILLAM WAY       │  2262 │
│ AK     │ ANCHORAGE │ 99515    │ 9600 MORNINGSIDE LOOP │  1740 │
│ AK     │ ANCHORAGE │ 99515    │ 9540 MORNINGSIDE LOOP │  1480 │
│ AK     │ JUNEAU    │ 99801    │ 2800 POSTAL WAY       │  1012 │
│ AK     │ PALMER    │ 99645    │ 1150 S COLONY WAY     │   897 │
│ AK     │ ANCHORAGE │ 99515    │ 9710 MORNINGSIDE LOOP │   860 │
│ AK     │ ANCHORAGE │ 99518    │ 7422 FOXRIDGE WAY     │   840 │
│ AK     │ ANCHORAGE │ 99515    │ 2100 MINERVA WAY      │   744 │
│ …      │ …         │ …        │ …                     │     … │
└────────┴───────────┴──────────┴───────────────────────┴───────┘

In [16]:
clean.group_by(
    "state",
    "city",
    "postcode",
    "street",
).agg(
    n=_.count(),
    units=_.unit.collect(),
).order_by(_.n.desc())

┏━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ state  ┃ city      ┃ postcode ┃ street                    ┃ n     ┃ units                               ┃
┡━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ string │ string    │ string   │ string                    │ int64 │ array<string>                       │
├────────┼───────────┼──────────┼───────────────────────────┼───────┼─────────────────────────────────────┤
│ AK     │ VALDEZ    │ 99686    │ 300 N HARBOR DRIVE        │   616 │ ['PIER A', 'PIER B', ... +613]      │
│ AK     │ ANCHORAGE │ 99515    │ 1200 W DIMOND BOULEVARD   │   523 │ ['MICRO', 'TRLR 1001', ... +520]    │
│ AK     │ ANCHORAGE │ 99504    │ 2221 MULDOON ROAD         │   473 │ ['TRLR 1', 'TRLR 10', ... +470]     │
│ AK     │ ANCHORAGE │ 99501    │ 905 RICHARDSON VISTA ROAD │   419 │ ['APT', 'APT 1', ... +417]          │
│ AK     │ ANCHORAGE │ 99508    │ 3240 PENLAND PARKWAY      │   391 │ ['TRLR 100', 'TRLR 101', ... +388]  │
│ AK     │ ANCHORAGE │ 99504    │ 5901 E 6TH AVENUE         │   359 │ ['TRLR 1', 'TRLR 10', ... +356]     │
│ AK     │ ANCHORAGE │ 99507    │ 9499 BRAYTON DRIVE        │   314 │ ['79', 'TRLR 1', ... +311]          │
│ AK     │ JUNEAU    │ 99801    │ 6590 GLACIER HIGHWAY      │   301 │ ['1', '10', ... +298]               │
│ AK     │ ANCHORAGE │ 99504    │ 1001 BONIFACE PARKWAY     │   221 │ ['TRLR 10E1', 'TRLR 10G', ... +218] │
│ AK     │ VALDEZ    │ 99686    │ 315 ATIGUN DRIVE          │   218 │ ['SPACE 1', 'SPACE 10', ... +215]   │
│ …      │ …         │ …        │ …                         │     … │ …                                   │
└────────┴───────────┴──────────┴───────────────────────────┴───────┴─────────────────────────────────────┘

In [10]:
def trigram_to_record_ids(t: ibis.Table) -> ibis.Table:
    counts_table = (
        t.select("record_id", t.trigrams.unnest().name("trigram"))
        .group_by("trigram")
        .agg(
            n_records=_.count(),
            record_ids=_.record_id.collect(),
        )
    )
    counts_table = counts_table.cache()
    return counts_table


trigrams_clean = trigram_to_record_ids(clean)
trigrams_dirty = trigram_to_record_ids(dirty)
trigrams_clean

┏━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ trigram ┃ n_records ┃ record_ids                   ┃
┡━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ string  │ int64     │ array<int64>                 │
├─────────┼───────────┼──────────────────────────────┤
│ STY     │       447 │ [122880, 245970, ... +445]   │
│ DIN     │      1269 │ [122880, 122881, ... +1267]  │
│ 572     │       703 │ [122880, 122881, ... +701]   │
│ 741     │       697 │ [122885, 247288, ... +695]   │
│ DRI     │     70660 │ [122891, 122892, ... +70658] │
│ 357     │       573 │ [122901, 123000, ... +571]   │
│ 662     │       421 │ [122912, 246592, ... +419]   │
│ NUG     │       193 │ [122912, 122914, ... +191]   │
│ 380     │      1263 │ [122916, 122970, ... +1261]  │
│ 391     │       735 │ [122919, 122972, ... +733]   │
│ …       │         … │ …                            │
└─────────┴───────────┴──────────────────────────────┘

In [19]:
def rare_trigrams(t: ibis.Table, threshold: int) -> ibis.Table:
    return t.filter(_.n_records < threshold)


trigrams_clean_rare = rare_trigrams(trigrams_clean, 100)
trigrams_dirty_rare = rare_trigrams(trigrams_dirty, 100)
trigrams_clean_rare

┏━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ trigram ┃ n_records ┃ record_ids                ┃
┡━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ string  │ int64     │ array<int64>              │
├─────────┼───────────┼───────────────────────────┤
│ JUS     │        64 │ [122968, 122970, ... +62] │
│ HSI     │         6 │ [247366, 247439, ... +4]  │
│ AEL     │        92 │ [247898, 247995, ... +90] │
│ HAE     │        73 │ [247898, 247995, ... +71] │
│ BUF     │        79 │ [248017, 248043, ... +77] │
│ RR-     │         2 │ [250529, 250530]          │
│ HEU     │         4 │ [250747, 250748, ... +2]  │
│ BUD     │        91 │ [250883, 268506, ... +89] │
│ NOX     │        54 │ [250894, 158607, ... +52] │
│ EYV     │        11 │ [251334, 251336, ... +9]  │
│ …       │         … │ …                         │
└─────────┴───────────┴───────────────────────────┘

In [20]:
join_table = ibis.join(
    trigrams_clean_rare,
    trigrams_dirty_rare,
    how="inner",
    predicates="trigram",
    lname="{name}_l",
    rname="{name}_r",
)
join_table

┏━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ trigram ┃ n_records_l ┃ record_ids_l              ┃ n_records_r ┃ record_ids_r              ┃
┡━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ string  │ int64       │ array<int64>              │ int64       │ array<int64>              │
├─────────┼─────────────┼───────────────────────────┼─────────────┼───────────────────────────┤
│ TYS     │          22 │ [122880, 296764, ... +20] │          54 │ [617538, 617539, ... +52] │
│ 09T     │           1 │ [210386]                  │          18 │ [990886, 969410, ... +16] │
│ 01S     │          10 │ [165789, 165790, ... +8]  │          30 │ [990935, 990936, ... +28] │
│ ONL     │          46 │ [196105, 229980, ... +44] │          15 │ [991126, 991127, ... +13] │
│ ILR     │          52 │ [297942, 299406, ... +50] │          34 │ [991377, 929079, ... +32] │
│ STS     │          23 │ [248491, 248536, ... +21] │          41 │ [991672, 991673, ... +39] │
│ HTI     │          30 │ [291302, 291323, ... +28] │          37 │ [991783, 991784, ... +35] │
│ EMB     │          55 │ [137194, 138812, ... +53] │          91 │ [992018, 992019, ... +89] │
│ YAW     │           2 │ [255632, 258411]          │           1 │ [992163]                  │
│ ALU     │          81 │ [261882, 261910, ... +79] │          73 │ [395637, 395638, ... +71] │
│ …       │           … │ …                         │           … │ …                         │
└─────────┴─────────────┴───────────────────────────┴─────────────┴───────────────────────────┘

In [26]:
links = join_table.select(
    "trigram",
    record_id_l=_.record_ids_l.unnest(),
    record_id_r=_.record_ids_r.unnest(),
)
print(links.count())
links

┌───────┐
│ 56920 │
└───────┘


┏━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ trigram ┃ record_id_l ┃ record_id_r ┃
┡━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ string  │ int64       │ int64       │
├─────────┼─────────────┼─────────────┤
│ TYS     │      122880 │      617538 │
│ TYS     │      296764 │      617539 │
│ TYS     │      203272 │      617540 │
│ TYS     │      203290 │      617541 │
│ TYS     │      203292 │      617542 │
│ TYS     │      203312 │      617543 │
│ TYS     │      203320 │      617544 │
│ TYS     │      203322 │      617545 │
│ TYS     │      203324 │      617546 │
│ TYS     │      122863 │      617547 │
│ …       │           … │           … │
└─────────┴─────────────┴─────────────┘

In [97]:
trigram_linkage = mismo.linkage.LinkTableLinkage(clean, dirty, links)
trigram_linkage = trigram_linkage.cache()
trigram_linkage

LinkTableLinkage(
    left=315_659,
    right=992_329,
    links=56_920,
)

In [98]:
trigram_linkage.links.with_left().with_right()

TypeError: LinksTable._swap_perspective() takes 1 positional argument but 2 were given

In [91]:
clean.count(), trigram_linkage.left.count(), trigram_linkage.left.with_n_links().count()

(┌────────┐
 │ 315659 │
 └────────┘,
 ┌────────┐
 │ 315659 │
 └────────┘,
 ┌────────┐
 │ 315659 │
 └────────┘)

In [93]:
trigram_linkage.left.with_n_links()

┏━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ record_id ┃ state  ┃ city         ┃ postcode ┃ street                 ┃ unit   ┃ lon         ┃ lat       ┃ trigrams                ┃ n_links ┃
┡━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━┩
│ int64     │ string │ string       │ string   │ string                 │ string │ float64     │ float64   │ array<string>           │ int64   │
├───────────┼────────┼──────────────┼──────────┼────────────────────────┼────────┼─────────────┼───────────┼─────────────────────────┼─────────┤
│         0 │ AK     │ ANCHOR POINT │ 99556    │ 23837 NIKOLAI STREET   │ NULL   │ -151.746357 │ 59.926617 │ ['STR', 'NIK', ... +20] │       0 │
│         1 │ AK     │ ANCHOR POINT │ 99556    │ 24338 MISTY LANE       │ NULL   │ -151.739071 │ 59.919594 │ ['ANE', 'LAN', ... +16] │       0 │
│         2 │ AK     │ ANCHOR POINT │ 99556    │ 24715 STERLING HIGHWAY │ NULL   │ -151.755646 │ 59.913984 │ ['WAY', 'GHW', ... +22] │       0 │
│         3 │ AK     │ ANCHOR POINT │ 99556    │ 24803 INLET COVE LANE  │ NULL   │ -151.757118 │ 59.912571 │ ['ANE', 'LAN', ... +18] │       0 │
│         4 │ AK     │ ANCHOR POINT │ 99556    │ 24825 INLET COVE LANE  │ NULL   │ -151.757657 │ 59.912205 │ ['ANE', 'LAN', ... +18] │       0 │
│         5 │ AK     │ ANCHOR POINT │ 99556    │ 24845 INLET COVE LANE  │ NULL   │ -151.757918 │ 59.911979 │ ['ANE', 'LAN', ... +18] │       0 │
│         6 │ AK     │ ANCHOR POINT │ 99556    │ 24865 INLET COVE LANE  │ NULL   │ -151.758861 │ 59.911740 │ ['ANE', 'LAN', ... +18] │       0 │
│         7 │ AK     │ ANCHOR POINT │ 99556    │ 25100 PEPPER ROAD      │ NULL   │ -151.709791 │ 59.908129 │ ['OAD', 'ROA', ... +17] │       0 │
│         8 │ AK     │ ANCHOR POINT │ 99556    │ 25238 STINKBUG STREET  │ NULL   │ -151.692157 │ 59.906734 │ ['BUG', 'TIN', ... +21] │       0 │
│         9 │ AK     │ ANCHOR POINT │ 99556    │ 25405 STINKBUG STREET  │ NULL   │ -151.695885 │ 59.903846 │ ['STR', 'BUG', ... +21] │       0 │
│         … │ …      │ …            │ …        │ …                      │ …      │           … │         … │ …                       │       … │
└───────────┴────────┴──────────────┴──────────┴────────────────────────┴────────┴─────────────┴───────────┴─────────────────────────┴─────────┘

In [94]:
trigram_linkage.left.with_n_links().n_links.topk(10)

┏━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ n_links ┃ n_links_count ┃
┡━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ int64   │ int64         │
├─────────┼───────────────┤
│       0 │        295003 │
│       1 │         17179 │
│       2 │          3158 │
│       3 │           279 │
│       4 │            39 │
│       5 │             1 │
└─────────┴───────────────┘

In [89]:
trigram_linkage.left.count()

┌────────┐
│ 315659 │
└────────┘

In [96]:
trigram_linkage.left.link_counts()

28164


┏━━━━━━━━━┳━━━━━━━━━━━┓
┃ n_links ┃ n_records ┃
┡━━━━━━━━━╇━━━━━━━━━━━┩
│ int64   │ int64     │
├─────────┼───────────┤
│       3 │       279 │
│       1 │     17179 │
│       0 │      7507 │
│       4 │        39 │
│   16486 │         0 │
│       5 │         1 │
│       2 │      3158 │
└─────────┴───────────┘

In [50]:
links_full = trigram_linkage.links.with_left().with_right()
links_full = links_full.select(sorted(links_full.columns))
links_full

┏━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┓
┃ city_l    ┃ city_r    ┃ lat_l     ┃ lon_l       ┃ postcode_l ┃ postcode_r ┃ record_id_l ┃ record_id_r ┃ state_l ┃ state_r ┃ street_l               ┃ street_r            ┃ trigram ┃ trigrams_l              ┃ trigrams_r              ┃ unit_l ┃ unit_r ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━┩
│ string    │ string    │ float64   │ float64     │ string     │ string     │ int64       │ int64       │ string  │ string  │ string                 │ string              │ string  │ array<string>           │ array<string>           │ string │ string │
├───────────┼───────────┼───────────┼─────────────┼────────────┼────────────┼─────────────┼─────────────┼─────────┼─────────┼────────────────────────┼─────────────────────┼─────────┼─────────────────────────┼─────────────────────────┼────────┼────────┤
│ ANCHORAGE │ ANCHORAGE │ 61.187445 │ -149.724311 │ 99504      │ 99504      │       49141 │      247399 │ AK      │ AK      │ 8400 RAGGED TOP CIRCLE │ 8321 RAGGED TOP CIR │ AGG     │ ['IRC', 'CIR', ... +18] │ ['CIR', 'GGE', ... +15] │ NULL   │ NULL   │
│ ANCHORAGE │ ANCHORAGE │ 61.187609 │ -149.724384 │ 99504      │ 99504      │       49155 │      247400 │ AK      │ AK      │ 8401 RAGGED TOP CIRCLE │ 8321 RAGGED TOP CIR │ AGG     │ ['IRC', 'CIR', ... +18] │ ['CIR', 'GGE', ... +15] │ NULL   │ NULL   │
│ ANCHORAGE │ ANCHORAGE │ 61.187424 │ -149.724106 │ 99504      │ 99504      │       49185 │      247401 │ AK      │ AK      │ 8410 RAGGED TOP CIRCLE │ 8321 RAGGED TOP CIR │ AGG     │ ['IRC', 'CIR', ... +18] │ ['CIR', 'GGE', ... +15] │ NULL   │ NULL   │
│ ANCHORAGE │ ANCHORAGE │ 61.187590 │ -149.724190 │ 99504      │ 99504      │       49196 │      247402 │ AK      │ AK      │ 8411 RAGGED TOP CIRCLE │ 8321 RAGGED TOP CIR │ AGG     │ ['IRC', 'CIR', ... +18] │ ['CIR', 'GGE', ... +15] │ NULL   │ NULL   │
│ ANCHORAGE │ ANCHORAGE │ 61.187404 │ -149.723910 │ 99504      │ 99504      │       49227 │      247485 │ AK      │ AK      │ 8420 RAGGED TOP CIRCLE │ 8340 RAGGED TOP CIR │ AGG     │ ['IRC', 'CIR', ... +18] │ ['CIR', 'GGE', ... +15] │ NULL   │ NULL   │
│ ANCHORAGE │ ANCHORAGE │ 61.187630 │ -149.723971 │ 99504      │ 99504      │       49244 │      247486 │ AK      │ AK      │ 8421 RAGGED TOP CIRCLE │ 8340 RAGGED TOP CIR │ AGG     │ ['IRC', 'CIR', ... +18] │ ['CIR', 'GGE', ... +15] │ NULL   │ NULL   │
│ ANCHORAGE │ ANCHORAGE │ 61.187387 │ -149.723743 │ 99504      │ 99504      │       49279 │      247487 │ AK      │ AK      │ 8430 RAGGED TOP CIRCLE │ 8340 RAGGED TOP CIR │ AGG     │ ['IRC', 'CIR', ... +18] │ ['CIR', 'GGE', ... +15] │ NULL   │ NULL   │
│ ANCHORAGE │ ANCHORAGE │ 61.187686 │ -149.723754 │ 99504      │ 99504      │       49291 │      247532 │ AK      │ AK      │ 8431 RAGGED TOP CIRCLE │ 8351 RAGGED TOP CIR │ AGG     │ ['IRC', 'CIR', ... +18] │ ['CIR', 'GGE', ... +15] │ NULL   │ NULL   │
│ ANCHORAGE │ ANCHORAGE │ 61.187371 │ -149.723576 │ 99504      │ 99504      │       49308 │      247533 │ AK      │ AK      │ 8440 RAGGED TOP CIRCLE │ 8360 RAGGED TOP CIR │ AGG     │ ['IRC', 'CIR', ... +18] │ ['CIR', 'GGE', ... +15] │ NULL   │ NULL   │
│ ANCHORAGE │ ANCHORAGE │ 61.187706 │ -149.723582 │ 99504      │ 99504      │       49316 │      247534 │ AK      │ AK      │ 8441 RAGGED TOP CIRCLE │ 8360 RAGGED TOP CIR │ AGG     │ ['IRC', 'CIR', ... +18] │ ['CIR', 'GGE', ... +15] │ NULL   │ NULL   │
│ …         │ …         │         … │           … │ …          │ …          │           … │           … │ …       │ …       │ …                      │ …                   │ …       │ …                     

In [46]:
links_full.mutate(jaccard=mismo.sets.jaccard(_.trigrams_l, _.trigrams_r)).order_by(
    _.jaccard.desc()
).filter(
    _.jaccard < 1,
    _.jaccard > 0.5,
)

┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┓
┃ city_l      ┃ city_r      ┃ lat_l     ┃ lon_l       ┃ postcode_l ┃ postcode_r ┃ record_id_l ┃ record_id_l_right ┃ record_id_r ┃ record_id_r_right ┃ state_l ┃ state_r ┃ street_l             ┃ street_r          ┃ trigram ┃ trigrams_l              ┃ trigrams_r              ┃ unit_l ┃ unit_r ┃ jaccard  ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━┩
│ string      │ string      │ float64   │ float64     │ string     │ string     │ int64       │ int64             │ int64       │ int64             │ string  │ string  │ string               │ string            │ string  │ array<string>           │ array<string>           │ string │ string │ float64  │
├─────────────┼─────────────┼───────────┼─────────────┼────────────┼────────────┼─────────────┼───────────────────┼─────────────┼───────────────────┼─────────┼─────────┼──────────────────────┼───────────────────┼─────────┼─────────────────────────┼─────────────────────────┼────────┼────────┼──────────┤
│ EAGLE RIVER │ EAGLE RIVER │ 61.277189 │ -149.371916 │ 99577      │ 99577      │      135092 │            135092 │      600173 │            600173 │ AK      │ AK      │ 6270 MAGNAVIEW DRIVE │ 6270 MAGNAVIEW DR │ GNA     │ ['VIE', 'IEW', ... +17] │ ['VIE', 'IEW', ... +16] │ NULL   │ NULL   │ 0.947368 │
│ ANCHORAGE   │ ANCHORAGE   │ 61.119973 │ -149.889473 │ 99515      │ 99515      │       83053 │             83053 │      406614 │            406614 │ AK      │ AK      │ 11150 SUKDU WAY      │ 11150 SUKDU       │ SUK     │ ['WAY', 'KDU', ... +15] │ ['KDU', 'UKD', ... +14] │ NULL   │ NULL   │ 0.941176 │
│ EAGLE RIVER │ EAGLE RIVER │ 61.258928 │ -149.466730 │ 99577      │ 99577      │      134972 │            134972 │      599755 │            599755 │ AK      │ AK      │ 4241 ELKHORN DRIVE   │ 4241 ELKHORN DR   │ KHO     │ ['ORN', 'LKH', ... +15] │ ['ORN', 'LKH', ... +14] │ NULL   │ NULL   │ 0.941176 │
│ ANCHORAGE   │ ANCHORAGE   │ 61.119973 │ -149.889473 │ 99515      │ 99515      │       83053 │             83053 │      406614 │            406614 │ AK      │ AK      │ 11150 SUKDU WAY      │ 11150 SUKDU       │ KDU     │ ['WAY', 'KDU', ... +15] │ ['KDU', 'UKD', ... +14] │ NULL   │ NULL   │ 0.941176 │
│ EAGLE RIVER │ EAGLE RIVER │ 61.309331 │ -149.552227 │ 99577      │ 99577      │      135979 │            135979 │      601878 │            601878 │ AK      │ AK      │ 9823 DINAAKA DRIVE   │ 9823 DINAAKA DR   │ AAK     │ ['NAA', 'AAK', ... +15] │ ['NAA', 'AAK', ... +14] │ NULL   │ NULL   │ 0.941176 │
│ EAGLE RIVER │ EAGLE RIVER │ 61.308055 │ -149.553310 │ 99577      │ 99577      │      135913 │            135913 │      601731 │            601731 │ AK      │ AK      │ 9706 DINAAKA DRIVE   │ 9706 DINAAKA DR   │ AAK     │ ['NAA', 'AKA', ... +15] │ ['NAA', 'AKA', ... +14] │ NULL   │ NULL   │ 0.941176 │
│ EAGLE RIVER │ EAGLE RIVER │ 61.258928 │ -149.466730 │ 99577      │ 99577      │      134972 │            134972 │      599763 │            599763 │ AK      │ AK      │ 4241 ELKHORN DRIVE   │ 4241 ELKHORN DR   │ LKH     │ ['ORN', 'LKH', ... +15] │ ['ORN', 'LKH', ... +14] │ NULL   │ NULL   │ 0.941176 │
│ ANCHORAGE   │ ANCHORAGE   │ 61.119973 │ -149.889473 │ 99515      │ 99515      │       83053 │             83053 │      406614 │            406614 │ AK      │ AK      │ 11150 SUKDU WAY      │ 11150 SUKDU       │ UKD     │ ['WAY', 'KDU', ... +15] │ ['KDU', 'UKD', ... +14] │ NULL   │ NULL   │ 0.941176 │
│ EAGLE RIVER │ EAGLE RIVER │ 61.254310 │ -149.2

In [43]:
linkage.link_counts_chart()

alt.HConcatChart(...)

In [8]:
linkage.links.count()

┌────────┐
│ 429374 │
└────────┘